# Multi-Input Multi-Output (MIMO)

In [ ]:
%pylab inline # to show plots within notebook

In [ ]:
"""
Multi-Input Multi-Output (MIMO) Channels
Single User

@author: Hassan Ghozlan
"""

from __future__ import division #makes float division default. for integer division, use //. e.g. 7//4=1

import matplotlib
#matplotlib.use('Agg') # force matplotlib to not use any Xwindows backend.
# fixes the error "no display name and no $DISPLAY environment variable"
# when code is run on Amazon Elastic Cloud Compute (EC2)

from numpy import *
from matplotlib.pyplot import *
from numpy.fft import ifft, fftshift, fft
from numpy.linalg import svd
from numpy.linalg import eigvalsh #eigvalsh : eigenvalues of a symmetric or Hermitian (conjugate symmetric)

from numpy.random import randint, standard_normal
from numpy.linalg import det
from numpy.linalg import inv # inverse
from numpy.linalg import pinv # psuedo-inverse

# MIMO Channel (Single User)

$$
\newcommand{\x}{\textbf{x}}
\newcommand{\y}{\textbf{y}}
\newcommand{\GM}{\textbf{G}}
\newcommand{\HM}{\textbf{H}}
\newcommand{\rv}{\textbf{r}}
\newcommand{\n}{\textbf{n}}
\newcommand{\z}{\textbf{z}}
\newcommand{\g}{\textbf{g}}
$$


The received signal is
$$ \rv = \HM \x + \n $$

# Optimal Receiver

The information rate of the **optimal** receiver is
$$ R_{\text{opt}} = \log(\det(\Sigma_r)) - \log(\det(\Sigma_{r|x})) $$
where
$$ \Sigma_r = \HM \Sigma_x \HM^\dagger + \Sigma_n $$
and
$$ \Sigma_{r|x} = \Sigma_x $$

In [ ]:
def info_rate_opt(H,Sigma_Z):
    Sigma_Y = ( H.dot(H.conj().transpose()) + Sigma_Z )
    Sigma_Y_X = ( Sigma_Z )
    R = log(det( Sigma_Y )) -  log(det( Sigma_Y_X ))
    
    return real(R)

# Linear Receiver

Consider a linear receiver. The processed received signal is
$$
\begin{align}
\y 
&= \GM \rv \\
&= \GM (\HM \x + \n) 
\end{align}
$$

The information rate is
$$
R_{\text{linear}} = \sum_{k=1}^{K} I(\x_{k}; \y_{k})
$$
We have
$$
I(\x_k; \y_k) = [ \log\det(\Sigma_{y,k}) - \log\det(\Sigma_{y|x,k})]
$$

where the covariance matrices are given by
$$
\begin{align}
\Sigma_{y,k} &= \GM_k (\HM \Sigma_{x} \HM^\dagger + \Sigma_{n,k}) \GM^\dagger_k \\
\Sigma_{y|x,k} &= \GM_k (\HM \bar{\Sigma}_{x,k} \HM^\dagger + \Sigma_{n,k}) \GM^\dagger_k \\
\end{align}
$$
where
$\bar{\Sigma}_{x,k}$ is the same as $\Sigma_{x}$ except that the entries involving symbols from **layer?** $k$ are set to $0$.


In [ ]:
def info_rate(G_RX,H,Sigma_Z):
    m, n = H.shape
    R_total = array(0)
    for i in range(n):
        G = G_RX[i,:].reshape((1,m))

        Sigma_X = identity(n)
        Sigma_Y = G.dot( 
        H.dot(Sigma_X).dot(H.conj().transpose()) + Sigma_Z 
        ).dot( G.conj().transpose() ) 
        
        Sigma_X_bar = Sigma_X
        Sigma_X_bar[i,i] = 0
        Sigma_Y_X = G.dot( 
        H.dot(Sigma_X_bar).dot(H.conj().transpose()) + Sigma_Z 
        ).dot( G.conj().transpose() ) 

        H_Y = sum(log(eigvalsh(Sigma_Y)))
        H_Y_X = sum(log(eigvalsh(Sigma_Y_X)))
        R = H_Y - H_Y_X    

        R_total = R_total + R
    return real(R_total)

* Minimum Mean Square Error (MMSE)
$$
\GM = \Sigma_x \HM^\dagger (\HM \Sigma_x \HM^\dagger + \Sigma_z)^{-1}
$$
* Zero-Forcing (ZF)
$$
\GM = \HM^{-1}
$$
* Matched Filter (MF)
$$
\GM = \HM^\dagger
$$

In [ ]:
def rx_matrix(H,RX):
    if RX == 'MF':
        G = H.conj().transpose()
    elif RX == 'ZF':
        G = inv(H)  
    elif RX == 'MMSE':
        G = (H.conj().transpose()).dot(inv(
        H.dot(H.conj().transpose()) + 
        Sigma_Z))
    return G

# Simulation

In [ ]:
# Generate a Rayleigh random variable (mean=0, variance=1)
def rayleigh(m,n):
    return sqrt(1/2) * (standard_normal((m,n)) + 1j * standard_normal((m,n)))

In [ ]:
NN = 2    
H = rayleigh(NN,NN)
SNR_dB = arange(-2,10,0.5)

R_ZEROS = zeros( len(SNR_dB) )
#RX_LIST = ['OPT','MMSE','ZF','MF']
RX_LIST = ['MMSE','ZF','MF']
RX = dict()
for rx in ['OPT']+RX_LIST: 
    RX[rx] = array(R_ZEROS)

for snr_index in range(len(SNR_dB)):
    print "SNR = " + str(SNR_dB[snr_index]) + " dB"
    SNR = 10**(SNR_dB[snr_index]/10)    #signal to noise ratio (linear)        
    sigma2 = 1.0/SNR            #noise variance

    Sigma_Z = sigma2 * identity(NN)

    R = info_rate_opt(H,Sigma_Z)
    RX['OPT'][snr_index] = R
    #print 'R = %.2f' %(R)
    
    for rx in RX_LIST:
        G_RX = rx_matrix(H, rx)
        R = info_rate(G_RX,H,Sigma_Z)        
        RX[rx][snr_index] = R
        #print 'R = %.2f' %(R)

In [ ]:
# Plot
SNR = 10**(SNR_dB/10)
R_SHANNON = log(1+SNR)
line_style = {'OPT':'-','MMSE':'.-','ZF':'x-','MF':'+-'}
for rx in ['OPT'] + RX_LIST:
    plot(SNR_dB,RX[rx],line_style[rx],label=rx)
xlabel('SNR (dB)')
ylabel('Rate (nats/symbol)')
legend(loc='upper left')